In [10]:
from script  import *
text = 'C:/Users/ASUS/AppData/Local/Temp/tmpa70zfck3.py'

vlayer = QgsVectorLayer(wards_shapefile, "airports", "ogr")
vlayer2 = QgsVectorLayer(lga_shapefile, "SHEET", "ogr")
vlayer3 = QgsVectorLayer(set_extent_shapefile, "SHEET", "ogr")

# for field in vlayer.fields():
#     print(field.name(), field.typeName())
# vlayer.displayField()

api ='5b3ce3597851110001cf62489ff9ae0b501e49cda73e83122c638e05'

location = geo_location(wards_shapefile,set_extent_shapefile, "Kwara", "Edu","Tsaragi II")
start = '4.544522,8.456104'
end = f"{location[0]},{location[1]}"
params = {"api_key":api, "start":start,"end":end}
link =f"https://api.openrouteservice.org/v2/directions/driving-car"
r = requests.get(link, params)
print(r.status_code)

404


In [4]:
processing.algorithmHelp("native:clip")
# parameters = { 'INPUT' : state_extent, 
#                'VALUE' : 1, 
#                 }
# point =processing.run('native:selectbylocation', parameters)
# /algs = dict()
# for alg in QgsApplication.processingRegistry().algorithms():
#     algs[alg.displayName()] = alg.id()
#     print(alg.displayName(), alg.id() )

Clip (native:clip)

This algorithm clips a vector layer using the features of an additional polygon layer. Only the parts of the features in the Input layer that fall within the polygons of the Overlay layer will be added to the resulting layer.

The attributes of the features are not modified, although properties such as area or length of the features will be modified by the clipping operation. If such properties are stored as attributes, those attributes will have to be manually updated.


----------------
Input parameters
----------------

INPUT: Input layer

	Parameter type:	QgsProcessingParameterFeatureSource

	Accepted data types:
		- str: layer ID
		- str: layer name
		- str: layer source
		- QgsProcessingFeatureSourceDefinition
		- QgsProperty
		- QgsVectorLayer

OVERLAY: Overlay layer

	Parameter type:	QgsProcessingParameterFeatureSource

	Accepted data types:
		- str: layer ID
		- str: layer name
		- str: layer source
		- QgsProcessingFeatureSourceDefinition
		- QgsProperty
	